In [2]:
!pip install langchain langchain-chroma langchain-openai
!pip install -U langchain-community tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 778.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:0

In [3]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [1]:
!pip install neo4j
from neo4j import GraphDatabase
driver = GraphDatabase.driver("neo4j+s://61c777.databases.neo4j.io", auth=("neo4j", "QGbjbBTN2Y2CldwzJc"))
uri =       "neo4j+s://61c777.databases.neo4j.io"
username =  "neo4j"
password =  "dWQGb5LujbBTN2Y2CldwzJc"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.1 MB/s eta 0:00:00


In [4]:
from langchain_core.documents import Document
import pandas as pd
#def Retrieving_expertise(disease):
def Retrieving_expertise():
 with GraphDatabase.driver(uri, auth=(username, password)) as driver:
  with driver.session() as session:
   #query = "MATCH (i:Expertise)--(c:Cluster)--(g:Gard) where g.GARDname=$disease RETURN i.Summarized_expertise as Summarized_expertise "
   #results = session.run(query, disease=disease)
   query = "MATCH (i:Expertise)--(c:Cluster) RETURN i.Summarized_expertise as Summarized_expertise "
   results = session.run(query)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   Data = pd.DataFrame(data)
   documents=[]
   for i in Data.index:
      documents.append(Document(page_content=Data['Summarized_expertise'][i]))
   return documents
A=['PGM1-CDG','Cerebral cavernous malformations 3',"CLOVES syndrome","STT3B-CDG","Thoracolaryngopelvic dysplasia","Actinic prurigo","Non-syndromic polydactyly","Genitopatellar syndrome","Diaphanospondylodysostosis"]
B=['GARD0004329','GARD0018314',"GARD0010939","GARD0017603","GARD0005184", "GARD0017510","GARD0004410","GARD0010994","GARD0016674"]
disease= A[0]
documents= Retrieving_expertise()
#documents= Retrieving_expertise(disease)


In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [9]:
results = vectorstore.similarity_search(" muscle weakness", 1)
for doc in results:
  print(doc.page_content)
  print("=================")

Exercise-Induced Muscle Fatigue


In [7]:
results = vectorstore.similarity_search_with_relevance_scores("muscle weakness")
for doc, score in results:
  print(score)
  print(doc.page_content)
  print("=================")

0.7793874423118192
Exercise-Induced Muscle Fatigue
0.7475369003047426
pgm1 deficiency and neuromuscular junction disorders
0.7386038757628586
Biochemical Analysis of Metabolic Myopathies
0.6814363329845914
ccm3 and mitochondrial pathway in neurological disease


In [10]:
def Retrieving_reserachers(results):
 documents=''
 for doc in results:
  Summarized_expertise=doc.page_content
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
   with driver.session() as session:
    query = "MATCH (i:Expertise)--(c:Cluster)--(r:Researcher)--(l:Location) where i.Summarized_expertise=$Summarized_expertise RETURN i.Summarized_expertise as Summarized_expertise, r.Name,r.Aff_name,l.Aff_country,l.Aff_city"
    results = session.run(query, Summarized_expertise=Summarized_expertise)
    data = [record.data() for record in results]  # Convert result to a list of dictionaries
    Data = pd.DataFrame(data)
    documents+='The following specialists work on '+ Summarized_expertise+': \n'
    for i in Data.index:
      documents+= str(i+1)+'-'+ Data['r.Name'][i]+ ' from ' + Data['r.Aff_name'][i].replace('[','').replace(']','') + ' which is in city of '+   Data['l.Aff_city'][i]   + ' and the country of '+   Data['l.Aff_country'][i] +'\n'
    return documents
#Summarized_expertise='maldi–tof/ms'
Retrieving_reserachers(results)

'The following specialists work on Exercise-Induced Muscle Fatigue: \n1-Marquardt T from Department of Pediatrics, University Hospital of Munster, Albert-Schweitzer-Campus 1 Gebaeude A13, 48149 Muenster, Germany. which is in city of Munster and the country of Germany\n2-Simic-Schleicher G from Klinikum Bremen Nord, Klinik für Kinder- und Jugendmedizin, Bremen, Germany. which is in city of Bremen and the country of Germany\n'

In [11]:
!pip install langchain langchain-chroma langchain-openai
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)
prompt = hub.pull("rlm/rag-prompt")
generation_chain = prompt | llm | StrOutputParser()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [30]:
message='I have a weak muscle'
def predict(message, _):
  documents=Retrieving_expertise()
  vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
  )
  results = vectorstore.similarity_search(message, 3)
  input_data = {"context": Retrieving_reserachers(results), "question": message}
  return generation_chain.invoke(input_data)
predict(message, _)

'You may benefit from consulting with specialists in Exercise-Induced Muscle Fatigue, such as Marquardt T from the University Hospital of Munster in Germany or Simic-Schleicher G from Klinikum Bremen Nord in Germany. They can provide guidance on improving muscle strength and function through tailored exercise programs. It is important to seek professional advice to address your weak muscle condition effectively.'

In [20]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 83.8 MB/s eta 0:00:00


In [31]:
import gradio as gr

gr.ChatInterface(predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Hi I am your Social Network virtual assistant, how can I help you today?", container=False, scale=7),
    title="SocialNetworkBot",
    theme="soft",
    examples=["I have muscle aches. What do you recommend?", "Which researchers specialize in back pain?"]  ).launch(share=True)

<ipython-input-31-30eced919232>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=300),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7968bc3027798b6c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Hugging face

In [32]:
!pip install huggingface_hub
!huggingface-cli loginy
from huggingface_hub import login
login(token ='token' )

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: argument {download,upload,repo-files,env,login,whoami,logout,auth,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag,version,upload-large-folder}: invalid choice: 'loginy' (choose from 'download', 'upload', 'repo-files', 'env', 'login', 'whoami', 'logout', 'auth', 'repo', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'scan-cache', 'delete-cache', 'tag', 'version', 'upload-large-folder')


# tools

In [35]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

In [69]:
from langchain_core.tools import tool
@tool
def find_expertise(message: str) -> str:
  "find related specialists"
  documents=Retrieving_expertise()
  vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
  )
  results = vectorstore.similarity_search(message, 3)
  print(message)
  input_data = {"context": Retrieving_reserachers(results), "question": 'Who can i Refer to treat '+ message}
  return generation_chain.invoke(input_data)

tools=[search,find_expertise]
#tools=[find_expertise]

def tool_chain(model_output):
  tool_map = {tool.name: tool for tool in tools}
  chosen_tool= tool_map[ model_output["name"] ]
  return  itemgetter("arguments") | chosen_tool

In [72]:
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate

rendered_tools= render_text_description(tools)
system_prompt= f""" you are an assistant that has access to the follwoing set of tools. Here are the name and description for each tool:
{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt=  ChatPromptTemplate.from_messages(
     [ ("system", system_prompt), ('user',"{input}") ]
)
print(prompt)
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

from operator import itemgetter

model= ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain= prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "I have a headache. Do you have any recommendations on how to relieve it?"})


input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=" you are an assistant that has access to the follwoing set of tools. Here are the name and description for each tool:\ntavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nfind_expertise(message: str) -> str - find related specialists\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


headache relief


'You can refer to Eva Morava-Kozicz from Mayo Clinic in Rochester, United States for headache relief.'

In [73]:
chain.invoke({"input": "What pills should I take to relieve a headache?"})

[{'title': 'Headache Medicines: Medications for Quick Relief - WebMD',
  'url': 'https://www.webmd.com/migraines-headaches/pain-relief-headaches',
  'content': "If you need headache relief, trying an over-the-counter (OTC) medicine is usually the first step. Aspirin, ibuprofen, acetaminophen, and naproxen sodium are different options you can try. Scientists are also studying how cognitive behavioral therapy (CBT) may help manage certain kinds of headaches. However, your headaches could be caused by another health condition that needs treatment. [...] acetaminophen (Tylenol)\nAspirin\nibuprofen (Advil/Motrin)\nnaproxen sodium (Aleve)\n\nPrescription medicine for headaches\nIf your headaches happen often or involve severe symptoms, then over-the-counter medicines might not work. Your doctor may try to identify what's causing your headaches and prescribe medication, such as:\n\n\ncelecoxib (Celebrex)\n\n\nfenoprofen (Nalfon)\n\n\nflurbiprofen (Ansaid)\n\n\nnabumetone (Relafen)\n\n\ndiclof

In [74]:
chain.invoke({"input": "In my research, I want to collaborate with new specialists in the back pain domain. Do you have any recommendations?"})

back pain


'You can refer to specialists such as Leslie G Biesecker, Palander O, Huang M, Singh SK, Singh CB, Grigelionis G, Grigelioniene G, and Hammarsjö A for the treatment of back pain. These specialists work in various countries such as Turkey, Canada, China, India, and Sweden.'